# Output Parsers Cơ Bản trong LangChain

## Giới thiệu

Output Parsers là các công cụ quan trọng trong LangChain giúp:
- **Chuẩn hóa** đầu ra từ LLM thành format mong muốn
- **Chuyển đổi** text response thành các cấu trúc dữ liệu Python
- **Xử lý lỗi** khi LLM trả về format không đúng
- **Tích hợp** dễ dàng với các hệ thống downstream

Trong notebook này, chúng ta sẽ tìm hiểu các parser cơ bản nhất.

## Setup môi trường

In [ ]:
# Import các thư viện cần thiết
from langchain_core.output_parsers import (
    StrOutputParser,
    CommaSeparatedListOutputParser,
    JsonOutputParser
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_anthropic import ChatAnthropic
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Khởi tạo model
model = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0,
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")
)

## 1. StrOutputParser - Parser đơn giản nhất

`StrOutputParser` chuyển đổi output của LLM thành string thuần túy. Đây là parser mặc định và đơn giản nhất.

In [ ]:
# Khởi tạo StrOutputParser
str_parser = StrOutputParser()

# Tạo prompt đơn giản
prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một nhà thơ. Hãy viết một câu thơ ngắn về {topic}.")
])

# Tạo chain: prompt -> model -> parser
chain = prompt | model | str_parser

# Chạy chain
result = chain.invoke({"topic": "mùa xuân"})

print("Kết quả (type):", type(result))
print("Nội dung:")
print(result)

In [ ]:
# So sánh với không dùng parser
chain_no_parser = prompt | model
result_no_parser = chain_no_parser.invoke({"topic": "mùa xuân"})

print("Không dùng parser:")
print("Type:", type(result_no_parser))
print("Content:", result_no_parser.content)
print("\nDùng StrOutputParser:")
print("Type:", type(result))
print("Content:", result)

## 2. CommaSeparatedListOutputParser

Parser này chuyển đổi text output thành Python list, với các items được phân tách bởi dấu phẩy.

In [ ]:
# Khởi tạo CommaSeparatedListOutputParser
list_parser = CommaSeparatedListOutputParser()

# Xem format instructions mà parser cung cấp
print("Format instructions:")
print(list_parser.get_format_instructions())
print("="*50)

In [ ]:
# Tạo prompt yêu cầu list output
list_prompt = ChatPromptTemplate.from_messages([
    ("system", """Bạn là một chuyên gia về {domain}.
    {format_instructions}"""),
    ("human", "Liệt kê 5 {items} phổ biến nhất.")
])

# Tạo chain với parser
list_chain = list_prompt | model | list_parser

# Test với các domain khác nhau
domains = [
    {"domain": "ẩm thực Việt Nam", "items": "món ăn"},
    {"domain": "lập trình", "items": "ngôn ngữ lập trình"},
    {"domain": "du lịch", "items": "điểm đến ở Việt Nam"}
]

for d in domains:
    result = list_chain.invoke({
        "domain": d["domain"],
        "items": d["items"],
        "format_instructions": list_parser.get_format_instructions()
    })
    
    print(f"\nDomain: {d['domain']}")
    print(f"Kết quả (type: {type(result)}):")
    for i, item in enumerate(result, 1):
        print(f"  {i}. {item}")

## 3. Xử lý lỗi với OutputParser

In [ ]:
# Test case có thể gây lỗi
error_prompt = ChatPromptTemplate.from_messages([
    ("human", "Viết một đoạn văn về {topic}")
])

# Chain với list parser (sẽ gặp lỗi nếu output không phải list)
error_chain = error_prompt | model | list_parser

try:
    result = error_chain.invoke({"topic": "biển"})
    print("Kết quả:", result)
except Exception as e:
    print(f"Lỗi xảy ra: {type(e).__name__}")
    print(f"Chi tiết: {str(e)}")
    print("\nĐây là lý do tại sao cần format instructions!")

## 4. Kết hợp nhiều Parser trong pipeline

In [ ]:
# Tạo một pipeline phức tạp hơn
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Custom function để xử lý list
def process_list(items):
    """Xử lý và format lại list items"""
    return {
        "count": len(items),
        "items": items,
        "formatted": "\n".join([f"• {item}" for item in items])
    }

# Prompt yêu cầu list
analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là chuyên gia phân tích."),
    ("human", """Liệt kê các bước để {task}.
    {format_instructions}""")
])

# Chain với multiple steps
analysis_chain = (
    analysis_prompt 
    | model 
    | list_parser
    | RunnableLambda(process_list)
)

# Test
result = analysis_chain.invoke({
    "task": "học lập trình Python từ cơ bản",
    "format_instructions": list_parser.get_format_instructions()
})

print(f"Số bước: {result['count']}")
print("\nDanh sách các bước:")
print(result['formatted'])

## 5. So sánh các Output Parser cơ bản

In [ ]:
# Prompt chung để test
test_prompt = ChatPromptTemplate.from_messages([
    ("human", "Liệt kê 3 loại trái cây nhiệt đới. {format_instructions}")
])

# Test với StrOutputParser
str_chain = test_prompt | model | StrOutputParser()
str_result = str_chain.invoke({"format_instructions": ""})

# Test với CommaSeparatedListOutputParser
list_chain = test_prompt | model | list_parser
list_result = list_chain.invoke({
    "format_instructions": list_parser.get_format_instructions()
})

print("=== StrOutputParser ===")
print(f"Type: {type(str_result)}")
print(f"Result: {str_result}")
print(f"\n=== CommaSeparatedListOutputParser ===")
print(f"Type: {type(list_result)}")
print(f"Result: {list_result}")
print(f"Length: {len(list_result)} items")

## 6. Use Case thực tế: Xử lý feedback từ người dùng

In [ ]:
# Scenario: Phân tích feedback và trích xuất keywords
feedback_prompt = ChatPromptTemplate.from_messages([
    ("system", """Bạn là chuyên gia phân tích feedback khách hàng.
    Nhiệm vụ: Trích xuất các từ khóa chính từ feedback.
    {format_instructions}"""),
    ("human", "Feedback: {feedback}")
])

# Chain để xử lý feedback
feedback_chain = feedback_prompt | model | list_parser

# Test với nhiều feedback
feedbacks = [
    "Sản phẩm rất tốt, giao hàng nhanh, đóng gói cẩn thận. Tuy nhiên giá hơi cao.",
    "Dịch vụ tệ, nhân viên không nhiệt tình, thời gian chờ lâu. Không hài lòng.",
    "App dễ sử dụng, giao diện đẹp, tính năng đầy đủ. Rất recommend!"
]

for fb in feedbacks:
    keywords = feedback_chain.invoke({
        "feedback": fb,
        "format_instructions": list_parser.get_format_instructions()
    })
    
    print(f"\nFeedback: {fb}")
    print(f"Keywords: {', '.join(keywords)}")

## 7. Custom processing với Output Parser

In [ ]:
# Tạo function để phân loại keywords
def categorize_keywords(keywords):
    """Phân loại keywords thành positive/negative"""
    positive_words = ['tốt', 'nhanh', 'đẹp', 'hài lòng', 'recommend', 'cẩn thận', 'dễ', 'đầy đủ']
    negative_words = ['tệ', 'cao', 'lâu', 'không', 'khó']
    
    result = {
        'positive': [],
        'negative': [],
        'neutral': []
    }
    
    for keyword in keywords:
        keyword_lower = keyword.lower()
        if any(pos in keyword_lower for pos in positive_words):
            result['positive'].append(keyword)
        elif any(neg in keyword_lower for neg in negative_words):
            result['negative'].append(keyword)
        else:
            result['neutral'].append(keyword)
    
    return result

# Enhanced chain với categorization
enhanced_chain = (
    feedback_prompt 
    | model 
    | list_parser
    | RunnableLambda(categorize_keywords)
)

# Test
test_feedback = "Sản phẩm chất lượng tốt, thiết kế đẹp nhưng giá khá cao và giao hàng chậm."
result = enhanced_chain.invoke({
    "feedback": test_feedback,
    "format_instructions": list_parser.get_format_instructions()
})

print(f"Feedback: {test_feedback}")
print("\nPhân tích:")
for category, keywords in result.items():
    if keywords:
        print(f"  {category.capitalize()}: {', '.join(keywords)}")

## 8. Tips và Best Practices

### 1. Luôn cung cấp format instructions
- Giúp LLM hiểu rõ format output mong muốn
- Giảm thiểu lỗi parsing

### 2. Xử lý lỗi gracefully
- Luôn có try-catch cho parsing operations
- Cung cấp fallback behavior

### 3. Chọn parser phù hợp
- `StrOutputParser`: Khi cần text thuần
- `CommaSeparatedListOutputParser`: Khi cần list đơn giản
- `JsonOutputParser`: Khi cần structured data (sẽ học ở bài sau)

In [ ]:
# Best practice example: Robust parsing với error handling
def safe_parse_list(prompt_template, input_data):
    """Parse list với error handling"""
    parser = CommaSeparatedListOutputParser()
    
    # Thêm format instructions vào input
    input_data['format_instructions'] = parser.get_format_instructions()
    
    try:
        # Tạo và chạy chain
        chain = prompt_template | model | parser
        result = chain.invoke(input_data)
        return {
            'success': True,
            'data': result,
            'error': None
        }
    except Exception as e:
        # Fallback: dùng StrOutputParser và split manually
        try:
            str_chain = prompt_template | model | StrOutputParser()
            str_result = str_chain.invoke(input_data)
            # Manual split
            items = [item.strip() for item in str_result.split(',')]
            return {
                'success': True,
                'data': items,
                'error': f'Fallback to manual parsing: {str(e)}'
            }
        except Exception as e2:
            return {
                'success': False,
                'data': [],
                'error': str(e2)
            }

# Test robust parsing
test_cases = [
    {"items": "framework Python"},
    {"items": "thư viện machine learning"}
]

robust_prompt = ChatPromptTemplate.from_messages([
    ("human", "Liệt kê 3 {items} phổ biến. {format_instructions}")
])

for test in test_cases:
    result = safe_parse_list(robust_prompt, test)
    print(f"\nTest: {test['items']}")
    print(f"Success: {result['success']}")
    if result['success']:
        print(f"Data: {result['data']}")
    if result['error']:
        print(f"Note: {result['error']}")

## Tổng kết

Trong notebook này, chúng ta đã học về:

1. **StrOutputParser**: Parser cơ bản nhất, chuyển đổi thành string
2. **CommaSeparatedListOutputParser**: Chuyển đổi thành Python list
3. **Format Instructions**: Cách hướng dẫn LLM output đúng format
4. **Error Handling**: Xử lý khi parsing thất bại
5. **Chaining**: Kết hợp parser với các processing steps khác
6. **Best Practices**: Robust parsing và fallback strategies

Output Parsers là thành phần quan trọng giúp tích hợp LLM vào ứng dụng thực tế. Trong các bài tiếp theo, chúng ta sẽ học về các parser phức tạp hơn như JsonOutputParser và PydanticOutputParser!